In [2]:

import time
import re

import rediscluster
import pymysql

import uuid

from transliterate import translit

import import_ipynb
from EnableCluster import *

In [25]:

sqlconn = pymysql.connect(host='localhost',user='root',password='password');
sqlconn.query('use monitor_vuz_2014;')
print(sqlconn)


In [26]:

rconn = rediscluster.RedisCluster(host='localhost',port='7000')
print(rconn)


RedisCluster<localhost:7000>


In [28]:

crs = sqlconn.cursor()
crs.execute('show tables;')
tables = crs.fetchall()

rconn.flushdb()

startT = time.time()
print(f'>>> START')

for table in tables:
 
    tabname = table[0]
    crs.execute('select * from '+tabname)
    tableRows = crs.fetchall()

    crs.execute('describe '+tabname)
    tableDescribe = crs.fetchall()
    tableColsNames = [r[0] for r in tableDescribe]

    for row in tableRows:
        rowsHashTableUUID = uuid.uuid4().hex

        keyVal = {}
        keyVal['TableName']=tabname
        for (n,d) in zip(tableColsNames,row):
            if d is None:
                d='None'
            elif type(d) == str:
                try:
                    d = translit(d,reversed=True)
                except:
                    pass
            keyVal[n]=d

        rconn.hset(str(rowsHashTableUUID),mapping=keyVal)
        rconn.rpush(tabname,rowsHashTableUUID)

endT = time.time()
print(f'<<< END : {(endT-startT):.3f} sec.')


>>> START
<<< END : 28.907 sec.


In [29]:
keys = rconn.keys()
for k in keys:
    r = re.search(r'^.{1,15}$',k.decode('ASCII'))
    if r != None:
        print(r.group(0))

okrug
department
ugs_vuz
form_org
stage
vuz
founder
region
effect
type_org
ugs
indicators


In [30]:
table_department = rconn.lrange('department',0,-1)
for d in table_department:
    print(d)

b'36b12f2c9cd242ad9eea833fd84383d2'
b'276e361de9574ef49dd90a1a58e96f94'
b'0204eb8807fb40a0b311a0323a64288c'
b'ad348781f25d401d98fb2ee5c9ae347e'
b'35c483d7d94f40ee892b2b2510067a8c'
b'b4d3ec0efb6c4453ac3665690ba11a34'
b'a1f1ed64c60641319479e5fbf5b4a4f2'
b'6eeeb309fcfe4fbab28a451857087b6a'
b'0d373722ddc742988d566d18f1cf8b5b'
b'c1e6ef59154f4292a6f8ff21f85ca68a'
b'7a8ab018aee74b438673e2902180b676'
b'8de37c1bdbbd4f568b26da4b1cc088ed'
b'dd8a93a4c2d440d89a920539fdcd2d4b'
b'216c6242bc6047d9a0e1e0bf97cad725'
b'96fa84de5bcb4063a64e9a82e1c6e133'
b'3e9d8b2397464adbbe6e411c0ded9147'
b'faf6c9863c334621a296e2bbc212054d'
b'433d78130f2e4236af07144eac68f466'
b'cb033fc85e184feda53fad464917642b'
b'62d0bc1ecd444daf8bdd98ec11b46762'
b'878932c7fc2f40eabfd1ed0fc8e29fd6'
b'c8b4517a09204d77979cf40170e3264d'
b'8866d62382bb47828c0f77bc1a12da23'
b'e857c35909c54426866e7a3fc383b8d1'
b'a490a7bbc57f406385c87de186b37677'
b'9f46954afa0b49aab75a5cde8ddb692a'
b'c43697b0a33b4918b37c070b9556d3cb'
b'd354aae328ce4f8c96436e32bd

In [31]:
rconn.hkeys(table_department[0])

[b'TableName', b'id', b'name']

In [32]:
rconn.hget(table_department[0],'TableName')

b'department'

In [33]:
rconn.hgetall(table_department[0])

{b'TableName': b'department',
 b'id': b'1',
 b'name': b"Federal'noe agentstvo zheleznodorozhnogo transporta "}